In [97]:
import healpy as hp
import numpy as np
import s2wav
import s2wav.filters as filters
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)

# Step 1: Read the Healpix map
healpix_map_path = 'data/planck_simulation/ffp10_newdust_total_030_full_map.fits'
healpix_map = hp.read_map(healpix_map_path, dtype=jnp.float64)

# Step 2: Convert Healpix map to spherical harmonic coefficients
L_max = hp.npix2nside(len(healpix_map))*2  # Define maximum multipole moment (choose appropriately based on your data)
alm = hp.map2alm(healpix_map, lmax=L_max-1)

# Step 3: Rearrange coefficients for s2wav
f = np.zeros((L_max, 2 * L_max - 1), dtype=np.complex128)

for l in range(L_max):
    for m in range(-l, l + 1):
        index = hp.Alm.getidx(L_max - 1, l, abs(m))
        if m < 0:
            f[l, L_max + m - 1] = (-1)**m * np.conj(alm[index])
        else:
            f[l, L_max + m - 1] = alm[index]

# Step 4: Perform wavelet analysis
N = 1  # Number of directional wavelets
filter = filters.filters_directional_vectorised(L_max, N)

# wavelet_coeffs, scaling_coeffs
f_wav, f_scal = s2wav.analysis(f, L_max, N, filters=filter)

# Step 5 Store the wavelet_coeffs, scaling_coeffs for future use
# f_wav is a list of jaxlib.xla_extension.ArrayImpl objects

# Convert each element in f_wav to a numpy array and save it
for i, wav in enumerate(f_wav):
    np_wav = np.array(wav)  # Convert JAX array to numpy array
    np.save(f"convolution/30/f_wav_{i}", np_wav)  

# Convert  f_scal to a numpy array and save it
np_scal = np.array(f_scal)  # Convert JAX array to numpy array
np.save(f"convolution/30/f_scal", np_scal)  

Stored_f_wav = [np.load(f"/Users/maxwang/Documents/projects/CMB_plot/convolution/30/f_wav_{i}.npy", allow_pickle=True) for i in range(len(f_wav))]
Sotred_f_sacl = np.load("/Users/maxwang/Documents/projects/CMB_plot/convolution/30/f_scal.npy")
# Step 6: reconstruct the signal if needed


KeyboardInterrupt: 

In [77]:
# f_wav_1 = f_wav.copy()

# for i, wav in enumerate(f_wav_1):
#     wav = np.array(wav)  # Convert JAX array to numpy array
#     # np.save(f"/convolution/30/f_wav_{i}.npy", np_wav)  # Save numpy array to a file


# for i in f_wav_1:
#     print(i.shape)


In [23]:

import healpy as hp
import numpy as np
import s2wav
import s2wav.filters as filters
import jax.numpy as jnp
import jax
Stored_f_wav = [np.load(f"/Users/maxwang/Documents/projects/CMB_plot/convolution/30/f_wav_{i}.npy", allow_pickle=True) for i in range(12)]


TypeError: bad number of pixels

In [94]:
# hp.npix2nside(len(healpix_map))
# hp.get_nside(healpix_map)

1024

In [105]:
healpix_map = hp.read_map(healpix_map_path, dtype=jnp.float64)
nside = hp.get_nside(healpix_map)
L_max = 2*nside+1

print(L_max)

2049


In [8]:
# !pip install mayavi
# https://groups.google.com/g/fatiando/c/gpFRXWabh_8?pli=1
# import mayavi

ModuleNotFoundError: No module named 'mayavi'

In [36]:
# pip install mlab
!pip install releases

In [39]:
import releases

ModuleNotFoundError: No module named 'releases'

In [42]:
import numpy as np
# import mayavi
from mayavi import mlab
# import mlab
import s2wav
import s2fft
from s2fft.sampling import s2_samples as samples


"""
Requirements:
    - s2fft
    - s2wav
    - numpy 
    - mayavi 
"""

def plot_sphere(f: np.ndarray, L: int, sr: float, mx: float, mn: float):

    # Define meshgrid points on spherical surface
    phis = samples.phis_equiang(L, sampling="mw")
    thetas = samples.thetas(L, sampling="mw")

    # Fix continuity at boundaries for visualisation
    thetas[0] = 0
    phis[-1] = 2 * np.pi

    # Generate angular meshgrid
    phi, theta = np.meshgrid(phis, thetas)

    # Scaling to increase/decrease magnitude of coefficient for visualisation
    temp = (f - mn) / mx
    r = sr + temp

    # Convert angular meshgrid to cartesian
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    # 3D render using mayavi package.
    mlab.figure(1, bgcolor=(1, 1, 1), fgcolor=(0, 0, 0), size=(300, 300))
    mlab.clf()
    mlab.mesh(x, y, z, scalars=temp, colormap="viridis",vmax=1-mn/mx,vmin=0)
    mlab.show()


# if __name__ == "__main__":
#     L = 128   # Harmonic bandlimit
#     N = 3     # Azimuthal bandlimit
#     sr = 5    # How much to scale the magnitude on the sphere for visualisation.

#     # Generate the wavelet kernel in spherical harmonic space
#     # Note: this generates the directional kernel, but the actual directionality is  
#     #       introduced during the convolution. So this is just a single set of harmonic
#     #       coefficients which you can imagine being rotated during the transform.
#     J_max = s2wav.samples.j_max(L)
#     wav_lmn, scal_lm = s2wav.filters.filters_directional_vectorised(L, N)

#     # Inverse spherical harmonic transform and plot the wavelet filters on the sphere.
#     for j in range(J_max):
#         wavelet_harmonic_coeffs = wav_lmn[j]
#         wavelet_coeffs = s2fft.inverse(wavelet_harmonic_coeffs, L)

#         mx, mn = np.nanmax(np.real(wavelet_coeffs)), np.nanmin(np.real(wavelet_coeffs))
#         plot_sphere(np.real(wavelet_pixel_space), L, sr, mx, mn)

In [47]:
L = 2049

plot_sphere(Stored_f_wav, L= 2049, sr = 5, mx= 1, mn = -1)

TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [109]:
healpix_map = hp.read_map(healpix_map_path, dtype=jnp.float64)
nside = hp.get_nside(healpix_map)
L_max = 2*nside+1


N = 4
filter = filters.filters_directional_vectorised(L_max, N)
f_wav, f_scal = s2wav.analysis(f = healpix_map, L = L_max, N = N, filters=filter, sampling= "healpix", nside = nside)


AssertionError: 

In [30]:
f_check = s2wav.synthesis(f_wav, f_scal, L_max, N, filters=filter)


In [88]:
# Step 7 store the reconstructed map
reconstructed_map = np.array(f_check)
np.save(f"/Users/maxwang/Documents/projects/CMB_plot/convolution/30/reconstructed_map", reconstructed_map)  # Save numpy array to a file
load_map = np.load("/Users/maxwang/Documents/projects/CMB_plot/convolution/30/reconstructed_map.npy")
# print(np.nanmean(np.abs(load_map-f)))

In [39]:
print(f"Mean absolute error = {np.nanmean(np.abs(f_check - f))}")

Mean absolute error = 2.0528992438406644e-07


In [89]:
# is_real = np.isreal(f)
# print(is_real.all()) 
#  setting reality=True in the context of spherical harmonic analysis indicates that the input data is real-valued, and the function leverages the conjugate symmetry property of the harmonic coefficients to optimize storage and computation.

False
